# Ribasim Functionality Demonstration

On this page we present a range of mostly synthetic models, set up to highlight individual
functionalities of Ribasim. The demonstrations are divided across two sections, in @sec-free
free-flowing basins are used, which can drain freely to a downstream basin. In @sec-level
level controlled basins are used. Here there is no free drainage, but water levels are
controlled by water management, such as pumping. Note that this is the only difference
between both, so all other functionality equally applies to free draining and level
controlled basins.

# Demonstration of Free Flowing LSW {#sec-free}

The following examples demonstrate the impact of forcings and the user allocation functionality for a single free draining LSW. The examples are simulated with synthetic data to highlight the functionality of Ribasim, with the exception of example @sec-free-balance and @sec-free-comparison, which show the Ribasim prototype simulation of the Hupsel LSW.

In [ ]:
import Pkg
Pkg.activate("../run")
VERSION

In [ ]:
using Ribasim
using Dates
using TOML
using Arrow
using DataFrames
import BasicModelInterface as BMI
using SciMLBase
using Graphs
using CairoMakie
using DataFrameMacros
using Revise

includet("../run/plot.jl")

In [ ]:
# Load LSW forcings data, used in several examples below
lswforcing = DataFrame(Arrow.Table("../test/data/lhm/forcing.arrow"));

## No external forcing {#sec-free-no}

This fictional free flowing LSW has no input or output forcing flux. The LSW empties according to the Volume-Area-Discharge relationship.
The two examples below show the impact the profile has on the LSW drainage.

In [ ]:
case = "emptying_sloping_profile"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_out = 2
id_lsw_end = 3
ids = [id_lsw, id_out, id_lsw_end]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2022-01-01")
config["endtime"] = Date("2022-02-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = DataFrame(; time=DateTime[], variable=Symbol[], id=Int[], value=Float64[])
config["node"] = DataFrame(; id=ids, node=["LSW", "OutflowTable", "LSW"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_out, "OutflowTable", "a"),
    (id_lsw, "LSW", "s") => (id_out, "OutflowTable", "s"),
    (id_out, "OutflowTable", "b") => (id_lsw_end, "LSW", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot results
plot_series(reg, id_lsw)

In [ ]:
case = "emptying_steep_profile_Hupsel"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_out = 2
id_lsw_end = 3
ids = [id_lsw, id_out, id_lsw_end]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2022-01-01")
config["endtime"] = Date("2022-02-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = DataFrame(; time=DateTime[], variable=Symbol[], id=Int[], value=Float64[])
config["node"] = DataFrame(; id=ids, node=["LSW", "OutflowTable", "LSW"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_out, "OutflowTable", "a"),
    (id_lsw, "LSW", "s") => (id_out, "OutflowTable", "s"),
    (id_out, "OutflowTable", "b") => (id_lsw_end, "LSW", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids,
    volume=[0.0, 7427.69, 14855.39, 750197.37],
    area=[92152.28, 92152.28, 92152.28, 92152.28],
    discharge=[0.0, 0.0, 0.096, 9.60],
    level=[18.90, 19.10, 19.30, 39.10])

## Simulate
reg = Ribasim.run(config)

## Plot results
plot_series(reg, id_lsw)

## Precipitation forcing

This fictional free flowing LSW is simulated with only the external forcing of synthetic precipitation data. The storage and the outflow respond to the preciptation as shown below.

In [ ]:
case = "precipitation"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_out = 2
id_lsw_end = 3
ids = [id_lsw, id_out, id_lsw_end]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2020-01-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = @subset(lswforcing, :variable == "P", :id == 1)
config["node"] = DataFrame(; id=ids, node=["LSW", "OutflowTable", "LSW"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_out, "OutflowTable", "a"),
    (id_lsw, "LSW", "s") => (id_out, "OutflowTable", "s"),
    (id_out, "OutflowTable", "b") => (id_lsw_end, "LSW", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

# Simulate
reg = Ribasim.run(config)

# Plot results
plot_series(reg, id_lsw)

## Evaporation forcing

The LSW loses water from evaporation. Outflow occurs according to the relation as in @sec-free-no, but overall volume decline is faster rate due to additional loss from evaporation. Evaporation does not occur in an empty LSW.

The second example shows the LSW with no simulated discharge, so that the only output is evaporation

In [ ]:
case = "evaporation"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_out = 2
id_lsw_end = 3
ids = [id_lsw, id_out, id_lsw_end]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-06-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = @subset(lswforcing, :variable == "E_pot", :id == 1)
config["node"] = DataFrame(; id=ids, node=["LSW", "OutflowTable", "LSW"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_out, "OutflowTable", "a"),
    (id_lsw, "LSW", "s") => (id_out, "OutflowTable", "s"),
    (id_out, "OutflowTable", "b") => (id_lsw_end, "LSW", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot results
plot_series(reg, id_lsw)

In [ ]:
case = "evaporation2"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_bnd = 2
ids = [id_lsw, id_bnd]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2020-01-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = @subset(lswforcing, :variable == "E_pot", :id == 1)
config["node"] = DataFrame(; id=ids, node=["LSW", "NoFlowBoundary"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_bnd, "NoFlowBoundary", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids=[1], volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot results
plot_series(reg, id_lsw)

## Evaporation and precipitation forcings

This example shows the evaporation and precipitation flux simulated together.

In [ ]:
case = "evaporation_precipitation"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_bnd = 2
ids = [id_lsw, id_bnd]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2020-01-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = @subset(lswforcing, :variable in ["P", "E_pot"], :id == 1)
config["node"] = DataFrame(; id=ids, node=["LSW", "NoFlowBoundary"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_bnd, "NoFlowBoundary", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids=[1], volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot
plot_series(reg, id_lsw)

## Infiltration 

Infiltration is an out flux of the LSW. This example shows the LSW storage responding to an enhanced forcing of infiltration. There is no outflow simulated of this LSW, so that the only output is the infiltration.

In [ ]:
case = "Infiltration"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_bnd = 2
ids = [id_lsw, id_bnd]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-02-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[id_lsw], variable=["infiltration"], value=[1.5e-1])
config["forcing"] = lswforcing[1:0, :]  # no forcing
config["node"] = DataFrame(; id=ids, node=["LSW", "NoFlowBoundary"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_bnd, "NoFlowBoundary", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids=[id_lsw], volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot results
plot_series(reg, id_lsw)

## Urban Runoff

Urban runoff is a surface water input to the LSW. In case of the Netherlands, this is not calculated by MODFLOW but by the unsaturated zone model MetaSWAP.
This example shows the LSW storage responding to the influx forcing of urban runoff.

In [ ]:
case = "Urban Runoff"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_bnd = 2
ids = [id_lsw, id_bnd]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-06-01")
config["state"] = DataFrame(; id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=[], variable=[], value=[])
config["forcing"] = @subset(lswforcing, :variable == "urban_runoff", :id == 1)
config["node"] = DataFrame(; id=ids, node=["LSW", "NoFlowBoundary"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_bnd, "NoFlowBoundary", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids=[id_lsw], volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot
plot_series(reg, id_lsw)

## Allocation to multiple users (agriculture and industry)

The allocation is based upon demand and prioritisation of the users and the available water in the LSW. In a free flowing LSW only water from the LSW can be abstracted by the users: agriculture and industry.
In this example there are two users. Agriculture has higher prioirty than industry, therefore when there is a shortage of available water, agriculture abstracts water before industry as demonstrated.

When water supply is limited, the model follows “de verdringingsreeks” (water prioritization rules in times of water shortage in the Netherlands).

<img src="https://user-images.githubusercontent.com/103200724/187085506-0ee79dd0-8023-42e8-9b05-4b9f433ae73e.jpg" alt="Multi user" title="Ribasim multi user allocation" width="800"/>

## Water balance of a single LSW (Hupsel) {#sec-free-balance}

This simulation is for the LSW Hupsel. The LSW is a free flowing LSW.

<img src="https://user-images.githubusercontent.com/103200724/187200598-edae91ad-8140-466a-b06e-3289babeee27.png" alt="Hupsel" title="Hupsel Water Balance" width="800"/>

## Water balance comparison Hupsel {#sec-free-comparison}

The following two figures show Hupsel LSW water balance for the Ribasim prototype compared to the water balance simulated by Mozart, the precursor to Ribasim.
The figures show a good agreement between the two simulations.

<img src="https://user-images.githubusercontent.com/4471859/179259174-0caccd4a-c51b-449e-873c-17d48cfc8870.png" alt="Mozart - Ribasim Water Balance Comparison" title="Mozart - Ribasim Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892581-557e2a0a-7b17-47cc-b65c-a2f7e3e30b66.PNG" alt="Mozart - Ribasim Storage Comparison" title="Mozart - Ribasim Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892581-557e2a0a-7b17-47cc-b65c-a2f7e3e30b66.PNG" alt="" title="Mozart - Ribasim Storage Comparison" width="800"/>

## Bifurcation

To handle bifurcations, we want to be able to model them using an adjustable fraction.
To implement this we use a separate `Bifurcation` node, that has an inflow `src`, and
outflows `dst_1` and `dst_2`. In this example `dst_1` will receive 33% of the flow, and
`dst_2` will receive `67%`. An error is raised if the fractions do not add up to 100%.

In [ ]:
case = "Bifurcation"

## Set up
config = Dict{String,Any}()
id_lsw = 1
id_out = 2
id_bif = 3
id_end1 = 4
id_end2 = 5
ids = [id_lsw, id_out, id_bif, id_end1, id_end2]
config["ids"] = ids
config["update_timestep"] = 86400.0
config["starttime"] = DateTime("2022-01-01")
config["endtime"] = DateTime("2022-02-01")
config["state"] = DataFrame(id=ids, S=1e6, C=0.1)
config["static"] = DataFrame(; id=id_bif, variable=["fraction_1", "fraction_2"], value=[0.33, 0.67])
config["forcing"] = DataFrame(; time=DateTime[], variable=Symbol[], id=Int[], value=Float64[])
config["node"] = DataFrame(; id=ids, node=["LSW", "OutflowTable", "Bifurcation", "LSW", "LSW"])
config["edge"] = Ribasim.edgepairs([
    (id_lsw, "LSW", "x") => (id_out, "OutflowTable", "a"),
    (id_lsw, "LSW", "s") => (id_out, "OutflowTable", "s"),
    (id_out, "OutflowTable", "b") => (id_bif, "Bifurcation", "src"),
    (id_bif, "Bifurcation", "dst_1") => (id_end1, "LSW", "x"),
    (id_bif, "Bifurcation", "dst_2") => (id_end2, "LSW", "x"),
])
config["profile"] = Ribasim.profilesets(;
    ids, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = Ribasim.run(config)

## Plot results
fig = Figure()
ylabel = "flow rate / m³ s⁻¹"
timespan = datetime2unix(config["starttime"]) .. datetime2unix(config["endtime"])
ax = time!(Axis(fig[1, 1]; ylabel), timespan.left, timespan.right)

name = Symbol(:bifurcation_, 3)
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊src₊Q)), label="bifurcation inflow")
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊dst_1₊Q)), label="bifurcation outflow 1")
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊dst_2₊Q)), label="bifurcation outflow 2")
fig[1, 2] = Legend(fig, ax, "", framevisible=true)
hidexdecorations!(ax, grid=false)
fig

# Demonstration of a Level Controlled LSW {#sec-level}

The following examples demonstrate the water management and user allocation functionality for a level controlled LSW. Polder de Tol is used as the example for these demonstrations.

## Single Level Controlled LSW (Tol)

This simulation is for the LSW De Tol. The LSW is a level controlled meaning that water is allocated to maintain water levels at a target level. 

<img src="https://user-images.githubusercontent.com/103200724/187208026-22b39c44-03c4-4694-8f82-79a0fa52551e.png" alt="tol" title="Tol Water Balance" width="800"/>




## De Tol water balance comparison with Mozart

The following two figures show De Tol LSW water balance for the Ribasim prototype compared to the water balance simulated by Mozart, the precursor to Ribasim.
The figures show a good agreement between the two simulations

<img src="https://user-images.githubusercontent.com/103200724/186892605-db217b55-13d6-48bb-a78e-3bbf4acc70c1.PNG" alt="Mozart - Ribasim Water Balance Comparison" title="Mozart - Ribasim Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892628-7cb0d3cd-9439-46cb-b6b8-9908d9c7e09a.PNG" alt="Mozart - Ribasim Water Balance Comparison" title="Mozart - Ribasim Water Balance Comparison" width="800"/>